In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import datetime as dt

In [3]:
udf = pd.read_csv('../data/2nd/tianchi_mobile_recommend_train_user_filtered.csv', index_col=['user_id', 'item_id'])
u = udf.reset_index()

In [8]:
def extract_result_set(df, date):
    return set(df[(df.date == date) & (df.behavior_type == 4)].index)

def f1(pred, ref):
    pred_len = float(len(pred))
    ref_len = float(len(ref))
    intersected_len = len(pred.intersection(ref))
    precision = intersected_len / pred_len
    recall = intersected_len / ref_len
    f = 2 * precision * recall / float(precision + recall)
    return f, precision, recall

def evaluate_method(case, date):
    prev_date = (dt.datetime.strptime(date, '%Y-%m-%d') - dt.timedelta(1)).strftime('%Y-%m-%d')
    predict_set = case(udf, prev_date)
    result_set = extract_result_set(udf, date)
    return f1(predict_set, result_set)

def date_range_evaluate_method_plot(case, date1, date2):
    selected_date_range = pd.date_range(date1, date2)
    analysis = pd.DataFrame(
        [list(evaluate_method(case, d.strftime('%Y-%m-%d'))) for d in selected_date_range],
        index=selected_date_range,
        columns=['f1', 'precision', 'recall']
    )
    analysis.plot(kind='bar', figsize=(15, 5))
    return analysis

In [13]:
def case2(df, date):
    cart_df = df[(df.date == date) & (df.behavior_type == 3) & (df.hour > 15)]
    buy_df = df[(df.date == date) & (df.behavior_type == 4)]
    
    filter_buy_df = cart_df[~cart_df.index.isin(buy_df.index)]
    
#     bought_lt18_set = set(df[(df.date == date) & (df.behavior_type == 4) & (df.hour <= 18)].reset_index().user_id)
#     bought_gt18_set = set(df[(df.date == date) & (df.behavior_type == 4) & (df.hour > 18)].reset_index().user_id)
#     filter_buy_df.reset_index(inplace=True)
    
    return set(filter_buy_df.index)

In [14]:
def case4(df, date):
    label_date = (dt.datetime.strptime(date, '%Y-%m-%d') + dt.timedelta(1)).strftime('%Y-%m-%d')
    features_df = pd.read_csv('../data/new2/apr23_train_%s_2nd_all_2.csv' % label_date, index_col=['user_id', 'item_id'])
    features_df = pd.merge(features_df.reset_index(),
                           df[['item_category']].reset_index().drop_duplicates(),
                           on=['user_id', 'item_id'], how='left'
                          ).set_index(['user_id', 'item_id'])
    
    sel = features_df[features_df.index.isin(case2(df, date))]
    
    def df18_selu(df):
        n = int(min(math.ceil(df.item_id.nunique() * 0.8), 5))
        del df['user_id']
        del df['item_category']
        return df.groupby(['item_id']).agg(dict(
                last_0_cart='first',
                itemCartToBuyRate1='first',
                itemCartToBuyRate2='first',
                last_0_collect='first',
                last_0_browse_times='first',
                #usercartToBuyRate1='first',
                #usercartToBuyRate2='first',
                last_1_cart='first',
                last_1_collect='first',
                last_1_browse_times='first',
                last_2_collect='first',
                last_2_cart='first',
                last_2_browse_times='first',
                item_beh4_at1='first',
                item_beh4_at2='first',
                item_beh4_at3='first'
            )).sort(['last_0_cart','itemCartToBuyRate1', 'itemCartToBuyRate2',
                     'item_beh4_at1','item_beh4_at2','item_beh4_at3',
                     'last_0_collect','last_0_browse_times',
                     # 'usercartToBuyRate1','usercartToBuyRate2',
                     'last_1_cart','last_1_collect','last_1_browse_times','last_2_cart','last_2_collect',
                     'last_2_browse_times'
                    ], ascending=False).head(n)
    selu = sel.reset_index().groupby(['user_id', 'item_category']).apply(df18_selu)
    
    rset = set(selu.reset_index().set_index(['user_id', 'item_id']).index)

    return rset

In [12]:
evaluate_method(case4, '2014-12-12')

(0.0521388790141012, 0.028888451185083056, 0.2671523982999393)

In [15]:
evaluate_method(case4, '2014-12-12')

(0.12388001558239188, 0.17282608695652174, 0.0965391621129326)